In [1]:
import pickle 
import os 
import pandas as pd 
import numpy as np
import h5py

In [2]:
checks_dir = '/cluster/home/prelotla/GitHub/projects2020_ohsu/peptides_generation/background'

path_mx = os.path.join(checks_dir, 'kmer_matrix_newmethod_correct')
path_pep = os.path.join(checks_dir, 'peptide_newmethod_output')

#test =h5py.File('/cluster/work/grlab/projects/projects2020_OHSU/spladder/TCGA_all_normals/genes_graph_conf2.merge_graphs.count.hdf5', 'r')
#all_strains = [strain.decode() for strain in test['strains'][:] ] 
sample_uniq = 'TCGA-06-0678-11A.all'
#idx_sample = [i for i, strain in enumerate(all_strains) if strain == sample_uniq][0]
idx_sample = 1

In [3]:
mx = pickle.load(open(path_mx, 'rb'))
segm_exp = 2
junc_exp = 3
kmers_i = 0 
is_junction = 1
pep = pickle.load(open(path_pep, 'rb'))

In [4]:
# Matrix mode used for multisample
seg_expr_mx = pd.DataFrame(mx[segm_exp])
seg_expr_mx['is_junction'] =  mx[is_junction]
seg_expr_mx = seg_expr_mx.rename({idx_sample: 'SegmExpr'}, axis = 1)
seg_expr_mx = seg_expr_mx.loc[:, ['is_junction', 'SegmExpr']]
seg_expr_mx.index = mx[kmers_i]


junc_exp_mx = pd.DataFrame(mx[junc_exp])
junc_exp_mx['is_junction'] =  mx[is_junction]
junc_exp_mx = junc_exp_mx.rename({idx_sample: 'JuncExpr'}, axis = 1)
junc_exp_mx = junc_exp_mx.loc[:, ['is_junction', 'JuncExpr']]
junc_exp_mx.index = mx[kmers_i]

mx_singles_sample = seg_expr_mx.join(junc_exp_mx.drop('is_junction', axis = 1))

In [5]:
mx_singles_sample.head()

,is_junction,SegmExpr,JuncExpr
MPAVKKEFP,False,0.59,NaN
PAVKKEFPG,False,0.59,NaN
AVKKEFPGR,False,0.59,NaN
VKKEFPGRE,False,0.59,NaN
KKEFPGRED,False,0.59,NaN


In [6]:
# Single output kmer object from single sample mode (same as Jiayu)
seg_expr_sample = [pep[i].segment_expr for i in np.arange(len(pep))]
junc_expr_sample = [pep[i].junction_expr for i in np.arange(len(pep))]
kmer_sample = [pep[i].kmer for i in np.arange(len(pep))]
is_junction_sample = [pep[i].is_cross_junction for i in np.arange(len(pep))]

mode_singles_sample = pd.DataFrame({'is_junction':is_junction_sample, 
                                   'SegmExpr':seg_expr_sample, 
                                    'JuncExpr':junc_expr_sample, 
                                   })
mode_singles_sample.index = kmer_sample

mode_singles_sample.head()

,is_junction,SegmExpr,JuncExpr
MPAVKKEFP,False,0.59,NaN
PAVKKEFPG,False,0.59,NaN
AVKKEFPGR,False,0.59,NaN
VKKEFPGRE,False,0.59,NaN
KKEFPGRED,False,0.59,NaN


In [7]:
mode_singles_sample.loc['WKRSIRHKG', :]

is_junction    True
SegmExpr       0.99
JuncExpr          2
Name: WKRSIRHKG, dtype: object

In [8]:
mypep = 'MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAALPPAASLPASAAGYEALLAPPLRPPRAYLSLHEAAPHLHLPRDPLALERFSATAAAAPDFQPLLDNGEPCIEVECGANRALLYVRKLCQGSKGPSIRHRGEWLTPNEFQFVSGRETAKDWKRSIRHKGSASALLAGKSLKTLMSKGILQVHPPICDCPGCRISSPV'

In [9]:
'KRSIRHKGK' in mypep

False

In [10]:
# 164 OK, then insertion of two kmers in the matrix!!! How is this possible ?
loc_kmer = 163
print('single')
display(mode_singles_sample.iloc[loc_kmer])
 
    
print('matrix')
display(mx_singles_sample.iloc[loc_kmer])
# position 165 and 166 artefac

single


is_junction    False
SegmExpr        0.59
JuncExpr         NaN
Name: DWKRSIRHK, dtype: object

matrix


is_junction    False
SegmExpr        0.59
JuncExpr         NaN
Name: DWKRSIRHK, dtype: object

In [11]:
mx_singles_sample.iloc[163:170, :]

,is_junction,SegmExpr,JuncExpr
DWKRSIRHK,False,0.59,NaN
WKRSIRHKG,True,0.99,2.0
KRSIRHKGS,True,1.58,2.0
RSIRHKGSA,True,2.18,2.0
SIRHKGSAS,True,2.77,2.0
IRHKGSASA,True,3.37,2.0
RHKGSASAL,True,3.96,2.0


In [12]:
'KRSIRHKGS' == 'KRSIRHKGS' 

True

In [13]:
mode_singles_sample.shape


(203, 3)

In [14]:
mx_singles_sample.shape

(203, 3)

In [15]:
mx_singles_sample.loc['WKRSIRHKG', :]

is_junction    True
SegmExpr       0.99
JuncExpr          2
Name: WKRSIRHKG, dtype: object

In [16]:
mx_singles_sample

,is_junction,SegmExpr,JuncExpr
MPAVKKEFP,False,0.59,NaN
PAVKKEFPG,False,0.59,NaN
AVKKEFPGR,False,0.59,NaN
VKKEFPGRE,False,0.59,NaN
KKEFPGRED,False,0.59,NaN
...,...,...,...
ICDCPGCRI,False,5.95,NaN
CDCPGCRIS,False,5.95,NaN
DCPGCRISS,False,5.95,NaN
CPGCRISSP,False,5.95,NaN


In [17]:
missing_kmers = [kmer for kmer in mx_singles_sample.index if kmer not in mode_singles_sample.index ]

In [18]:
missing_kmers

[]

In [19]:
mx_singles_sample_short = mx_singles_sample.drop(missing_kmers, axis = 0)

In [20]:
compare = mx_singles_sample_short.merge(mode_singles_sample ,  left_index=True,
                                      right_index=True)

In [21]:
compare.head()

,is_junction_x,SegmExpr_x,JuncExpr_x,is_junction_y,SegmExpr_y,JuncExpr_y
MPAVKKEFP,False,0.59,NaN,False,0.59,NaN
PAVKKEFPG,False,0.59,NaN,False,0.59,NaN
AVKKEFPGR,False,0.59,NaN,False,0.59,NaN
VKKEFPGRE,False,0.59,NaN,False,0.59,NaN
KKEFPGRED,False,0.59,NaN,False,0.59,NaN


In [22]:
foo = compare['SegmExpr_x'] - compare['SegmExpr_y']

In [23]:
foo2 = foo[~np.isnan(foo)]
foo3 = foo[foo !=0]

In [24]:
compare.loc[foo3.index]

,is_junction_x,SegmExpr_x,JuncExpr_x,is_junction_y,SegmExpr_y,JuncExpr_y


In [25]:
foo3

Series([], dtype: float64)

In [26]:
foo = compare['JuncExpr_x'] - compare['JuncExpr_y']

In [27]:
foo[~np.isnan(foo)]

WKRSIRHKG    0.0
KRSIRHKGS    0.0
RSIRHKGSA    0.0
SIRHKGSAS    0.0
IRHKGSASA    0.0
RHKGSASAL    0.0
HKGSASALL    0.0
KGSASALLA    0.0
GSASALLAG    0.0
dtype: float64

In [28]:
mx_singles_sample.loc[missing_kmers, :]

,is_junction,SegmExpr,JuncExpr


In [135]:
'KRSIRHKGK' in mode_singles_sample.index

False

In [129]:
missing_kmers

['KRSIRHKGK', 'RSIRHKGKV']

In [128]:
mx_singles_sample.tail()


,is_junction,SegmExpr,JuncExpr
ICDCPGCRI,False,0.7,NaN
CDCPGCRIS,False,0.7,NaN
DCPGCRISS,False,0.7,NaN
CPGCRISSP,False,0.7,NaN
PGCRISSPV,False,0.7,NaN


In [ ]:
# JUNCTION EXPRESSION LOOKS GOOD - BUT SEGMENT EXPRESSION IS NOT 
# + THEY ARE TWO MISSING KMERS